# GUIA 4

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats

### Ejercicio 1

![image.png](enunciados/ej1.png)

a)

Del enunciado se tienen los siguientes datos

$\tau$: Precision de Lucas

$D$: Punto de impacto del tiro de Lucas 

$X$: Coordenada del punto de impacto del tiro de lucas

$$X|_{\tau} \sim \mathcal{N}(0, \frac{1}{\tau})$$

$$\tau \sim \chi^2_8$$

$$D \sim U(X, 0)$$

$$\sum_{i=1} ^{10} x_i^{2} = 17 $$


Queremos calcular la distribucion a posteriori $p(\tau | x)$, es decir, la precision de Lucas luego de que hemos observado los datos de sus 10 tiros (la sumatoria).

Por Bayes tenemos que 

$$ p(\tau | x) \propto p(x | \tau) \cdot p(\tau)$$


Comenzamos calculando $ p(x | \tau) $

$p(x_i | \tau) = \frac{\sqrt{\tau}}{\sqrt{2 \pi}} \exp (-\frac{\tau}{2} x_i^{2})$

Si multiplicamos para n=10 (ya que observamos 10 tiros)

$p(x | \tau) = \prod_{i=1} ^{10} p(x_i | \tau) =\left ( \frac{\sqrt{\tau}}{\sqrt{2 \pi}} \right) ^{10} \cdot \exp( -\frac{\tau}{2} \sum_{i=1} ^{10}  x_i^{2})$

Ahora calculamos $p(\tau)$, que como sabemos que $\tau \sim \chi^2_8$, queda


$p(\tau) = \frac{1}{2^{8/2} \Gamma(\frac{8}{2})} \tau^{\frac{8}{2}-1} \exp{(-\frac{\tau}{2})} \mathbb{1}\{x>0\} $

$ p(\tau) = \frac{1}{96} \tau^3 \exp{(- \frac{\tau}{2})} \mathbb{1}\{x>0\}  $


Reemplazando en la formula de Bayes


$ p(\tau | x) \propto \left ( \frac{\sqrt{\tau}}{\sqrt{2 \pi}} \right) ^{10} \cdot \exp( -\frac{\tau}{2} \sum_{i=1} ^{10}  x_i^{2}) \cdot \frac{1}{96} \tau^3 \exp{(- \frac{\tau}{2})} \mathbb{1}\{x>0\} $



$ p(\tau | x) \propto \left ( \frac{\sqrt{\tau}}{\sqrt{2 \pi}} \right) ^{10} \cdot \exp( -\frac{\tau}{2} 17) \cdot \frac{1}{96} \tau^3 \exp{(- \frac{\tau}{2})} \mathbb{1}\{x>0\} $

$ p(\tau | x) \propto \alpha \cdot \tau^8 \cdot \exp(-9\tau) \mathbb{1}\{x>0\}$

Esta ultima expresion fue "acomodada" para parecerse a una distribucion $\Gamma(\nu=9, \lambda=9)$, por lo que es necesario que $\alpha = \frac{9^9}{\Gamma(9)}$


De este modo, 

$$ \tau|x \sim \Gamma(\nu=9, \lambda=9) $$









b) la distribucion predictiva describe el comportamiento de una nueva observacion, teniendo en cuenta lo calculado anteriormente.

Como vimos, la distribucion a posteriori de $\tau$ es $ \tau|x \sim \Gamma(\nu=9, \lambda=9) $.

Como lo indica el enunciado, una nueva observacion $X_n$ dado que se conoce el parametro $\tau$ seguira teniendo una distribucion normal al igual que X, por lo que se sigue cumpliendo que

$$ X_n | \tau  \sim \mathcal{N}(0, \frac{1}{\tau}) $$

Ahora, la distribucion predictiva de $X_n$ es la integral de la distribucion condicional de $X_n$, multiplicada por la posteriori de $\tau$


$$ p(X_n | x) = \int _{0} ^{\infty} p(X_n | \tau) \cdot p(\tau | x)  d\tau$$


Calculando por partes

$ p(X_n | \tau) = \frac{1}{\sqrt{\frac{2 \pi}{\tau}}} \exp(-\frac{\tau X_n^{2}}{2} ) $ 

$ p(\tau | x) = \frac{9^9}{\Gamma(9)} \tau^8 e^{-9 \tau} $

Reemplazando en la integral


$ p(X_n | x) = \int _{0} ^{\infty} \frac{1}{\sqrt{\frac{2 \pi}{\tau}}} \exp(-\frac{\tau X_n^{2}}{2}) \cdot \frac{9^9}{\Gamma(9)} \tau^8 e^{-9 \tau} d\tau$

$  p(X_n | x) = \frac{9^9}{\Gamma(9)} \cdot \frac{1}{\sqrt{2\pi}} \int _{0} ^{\infty} \tau^{\frac{17}{2}} \cdot \exp{(-\tau (\frac{X_n^{2}}{2}+ 9))} d\tau $


Observando con detenimiento, la expresion es muy similar a una distribucion Gamma

$ p(X_n | x) = \frac{9^9}{\Gamma(9) \sqrt{2\pi}} \cdot \frac{1}{(X_n^{2} + 9)^\frac{19}{2}} $

Reacomodando los terminos se puede llegar a una expresion como

$ p(X_n | x) = \frac{\Gamma(\frac{19}{2})}{\Gamma(9) \sqrt{18\pi}} \cdot (\frac{X_n^{2}}{18} + 1 )^{-\frac{19}{2}}$ 


Finalmente, se compara con la densidad de la t-Student:


$f(t \mid \nu, \mu, \sigma) = \frac{\Gamma\left(\frac{\nu + 1}{2}\right)}{\sqrt{\nu\pi} \, \Gamma\left(\frac{\nu}{2}\right) \sigma} \left(1 + \frac{1}{\nu}\left(\frac{t - \mu}{\sigma}\right)^2\right)^{-\frac{\nu + 1}{2}}$

Se puede ver que $X_n$ tiene una distribucion del tipo t-Student, cuyos parametros son $\nu = 18$ grados de libertad, media $\mu =0 $, y escala $\sigma = 1$.

$X_n | x \sim t_{18}(0, 1)$





In [4]:
# c)

# Defino los grados de libertad
df = 18

# P(|X|<2.1 | x) = P(-2.1 < X < 2.1 | x)

# Calculo la probabilidad acumulada en los límites -2.1 y 2.1 
p_i = stats.t.cdf(-2.1, df)
p_d = stats.t.cdf(2.1, df)

# La probabilidad de que el próximo tiro esté entre -2.1 y 2.1 
p_tot = p_d - p_i
print(f'La probabilidad de que el próximo tiro esté entre -2.1 y 2.1 es {p_tot:.4f}')



La probabilidad de que el próximo tiro esté entre -2.1 y 2.1 es 0.9499
